#Starting with I/O the data set the first subsets of each data

Changed the file type to .Xls for file uploading, and imported on a colab upload to drive.

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving company_x_dev_modeled_metrics.xls to company_x_dev_modeled_metrics.xls
Saving company_x_dev_reported_metrics.xls to company_x_dev_reported_metrics.xls
Saving company_x_prod_modeled_metrics.xls to company_x_prod_modeled_metrics.xls
Saving company_x_prod_reported_metrics.xls to company_x_prod_reported_metrics.xls


In [ ]:
import io
df_modeled_dev = pd.read_excel(io.BytesIO(uploaded['company_x_dev_modeled_metrics.xls']))
df_report_prod = pd.read_excel(io.BytesIO(uploaded['company_x_prod_reported_metrics.xls']))
df_report_dev = pd.read_excel(io.BytesIO(uploaded['company_x_dev_reported_metrics.xls']))
df_modeled_prod = pd.read_excel(io.BytesIO(uploaded['company_x_prod_modeled_metrics.xls']))


## Date Quality and Validation: Starting in Dev Reports
#### Checking for the columns shape, amount to get a look at the data frame.

In [ ]:
print("num of rows and columns",df_report_dev.shape)
print("Nulls by columns", df_report_dev.isnull().sum())
print("non null vals", df_report_dev.notnull().sum())
df_report_dev.head()

num of rows and columns (16383, 6)
Nulls by columns REPORTED_DATE        0
COMPANY              0
MARKETING_CHANNEL    0
CAMPAIGN_ID          0
METRIC_NAME          0
METRIC_VALUE         0
dtype: int64
non null vals REPORTED_DATE        16383
COMPANY              16383
MARKETING_CHANNEL    16383
CAMPAIGN_ID          16383
METRIC_NAME          16383
METRIC_VALUE         16383
dtype: int64


,REPORTED_DATE,COMPANY,MARKETING_CHANNEL,CAMPAIGN_ID,METRIC_NAME,METRIC_VALUE
0,2024-07-01,COMPANY_X,ADWORDS,21167876130,CHANNEL_REPORTED_REVENUE,2.846547
1,2024-07-01,COMPANY_X,FACEBOOK_ADS,6281663656977,IMPRESSIONS,1384.000000
2,2024-07-01,COMPANY_X,FACEBOOK_ADS,6281654088577,REACH,2063.000000
3,2024-07-01,COMPANY_X,FACEBOOK_ADS,6281663656977,CPC,1.482045
4,2024-07-01,COMPANY_X,FACEBOOK_ADS,6281654088577,CLICKS,35.000000


#### Based on shape of df - data set will need to be transformed on the metric name to get the insights of the data set and get the new shape
- step 1 get the metic name column values to see how much and what ways to get the new shape of the data

In [ ]:
df_report_dev["METRIC_NAME"].unique()


array(['CHANNEL_REPORTED_REVENUE', 'IMPRESSIONS', 'REACH', 'CPC',
       'CLICKS', 'CTR', 'CONVERSIONS', 'SPEND', 'CVR', 'FREQUENCY', 'CPM',
       'CLICKTHROUGHS'], dtype=object)

##Next steps some data processing:
### Created a function to proccess the excel files to the correct shape for analysis
- First with a pivot tables to transfor the data set where metric names are as columns in reported dev and prod
- Secondly to check the data quality of how each proposed data set will compare to each other

In [ ]:
# based on the data creating a function to repeat this process would be helpfull
'''
Pivot the table by metric name to correlctly assess the shape, dtype,null and non null values of the data frame (reported_metrics)
'''

def df_pivot_data(df_input):
  df_pivoted = df_input.pivot_table(
   index=["REPORTED_DATE", "COMPANY", "MARKETING_CHANNEL", "CAMPAIGN_ID"],
   columns = "METRIC_NAME",
   values= "METRIC_VALUE",
   aggfunc='first'
    ).reset_index()
  return df_pivoted


'''
get the summary of a data set of the dtype, nulls, non nulls, percentage of nulls , descrite data unique values
'''
def check_data_summary(df_input):
  # creates df_summary
  contain_summary = []

  for df_column in df_input.columns:
    datatype = df_input[df_column].dtype
    null_amount = df_input[df_column].isnull().sum()
    value_count = df_input[df_column].notnull().sum()
    distinct_val = df_input[df_column].nunique()
    duplicate_val = df_input[df_column].duplicated().sum()

    #check if the column is descrite by unique values just to get
    #the amount of distinct values or the potential categroical/ordinal vals
    #if distinct_val < 10:
    #  distinct_type = df_input[df_column].unique()
    #else:
    #  distinct_type = distinct_val

    # dict to conver to df
    contain_summary.append({
        "column_name": df_column,
        "data_type" : datatype,
        "null_sum"  :  null_amount,
        "value_sum" : value_count,
        "distinct_type": distinct_val,
        "duplicate_sum": duplicate_val
        #add in the duplicates
    })

  summary_df = pd.DataFrame (contain_summary)

  # pct of nulls by columns
  summary_df['percentage_of_null'] = ((summary_df['null_sum'] / (summary_df['value_sum'] + summary_df['null_sum']) )* 100).round(2)

  return summary_df

## dev displayed in the example for the pivot tables
report_dev_pivot = df_pivot_data(df_report_dev)
report_dev_summary = check_data_summary(report_dev_pivot)
display(report_dev_summary)



,column_name,data_type,null_sum,value_sum,distinct_type,duplicate_sum,percentage_of_null
0,REPORTED_DATE,datetime64[ns],0,2701,36,2665,0.00
1,COMPANY,object,0,2701,1,2700,0.00
2,MARKETING_CHANNEL,object,0,2701,7,2694,0.00
3,CAMPAIGN_ID,int64,0,2701,90,2611,0.00
4,CHANNEL_REPORTED_REVENUE,float64,187,2514,1137,1563,6.92
5,CLICKS,float64,46,2655,491,2209,1.70
6,CLICKTHROUGHS,float64,2603,98,54,2646,96.37
7,CONVERSIONS,float64,247,2454,254,2446,9.14
8,CPC,float64,2203,498,464,2236,81.56
9,CPM,float64,2285,416,416,2284,84.60


## Dev Quality Check:
#### Now I see that theres is a high percentage of nulls in metrics for click throughs, cpm, frequency, cpc and cvr , and reach.

 - Based in DEV , and comparing it to prod I want to see the comparsion between the two because (how is the weight of each metric weight for  reveune is the same or different?)

- How are the differences in duplicats, percentages of nulls, effecting distributions ?

- How does this play on model preformance ?

## Report Prod Check:
### Now checking in prods reported data quality check

In [ ]:
print(df_report_prod.head(2))
pivoted_report_prod = df_pivot_data(df_report_prod)
print(pivoted_report_prod.head(2))
report_prod_summary = check_data_summary(pivoted_report_prod)
display(report_prod_summary )


  REPORTED_DATE    COMPANY MARKETING_CHANNEL      CAMPAIGN_ID METRIC_NAME  \
0    2024-07-01  COMPANY_X        STACKADAPT           430328      CLICKS   
1    2024-07-01  COMPANY_X        AMAZON_ADS  374998198609962       SPEND   

   METRIC_VALUE  
0        169.00  
1        400.07  
METRIC_NAME REPORTED_DATE    COMPANY MARKETING_CHANNEL  CAMPAIGN_ID  \
0              2024-07-01  COMPANY_X           ADWORDS  14731096318   
1              2024-07-01  COMPANY_X           ADWORDS  16878784202   

METRIC_NAME  CHANNEL_REPORTED_REVENUE  CLICKS  CLICKTHROUGHS  CONVERSIONS  \
0                             1873.13    13.0            NaN          5.0   
1                                0.00    15.0            NaN          NaN   

METRIC_NAME  CPC  CPM       CTR       CVR  FREQUENCY  IMPRESSIONS  REACH  \
0            NaN  NaN  3.357646  3.521127        NaN        128.0    NaN   
1            NaN  NaN  3.772018  0.000000        NaN         97.0    NaN   

METRIC_NAME   SPEND  
0            190.

,column_name,data_type,null_sum,value_sum,distinct_type,duplicate_sum,percentage_of_null
0,REPORTED_DATE,datetime64[ns],0,2701,36,2665,0.00
1,COMPANY,object,0,2701,1,2700,0.00
2,MARKETING_CHANNEL,object,0,2701,7,2694,0.00
3,CAMPAIGN_ID,int64,0,2701,90,2611,0.00
4,CHANNEL_REPORTED_REVENUE,float64,187,2514,1137,1563,6.92
5,CLICKS,float64,45,2656,492,2208,1.67
6,CLICKTHROUGHS,float64,2603,98,54,2646,96.37
7,CONVERSIONS,float64,247,2454,253,2447,9.14
8,CPC,float64,2203,498,464,2236,81.56
9,CPM,float64,2285,416,416,2284,84.60


# Producing Graphs to show the differences for Comparsion across columns for Prod vs Dev reports

Adding in a comparsion between both visually to see if theres a difference between these metrics in a bar plot

In [ ]:
report_prod_summary["env"] = "prod"
report_dev_summary["env"] = "dev"
report_summary_combined = pd.concat([report_prod_summary, report_dev_summary], ignore_index=True)

report_summary_combined['column_name'].unique()

array(['REPORTED_DATE', 'COMPANY', 'MARKETING_CHANNEL', 'CAMPAIGN_ID',
       'CHANNEL_REPORTED_REVENUE', 'CLICKS', 'CLICKTHROUGHS',
       'CONVERSIONS', 'CPC', 'CPM', 'CTR', 'CVR', 'FREQUENCY',
       'IMPRESSIONS', 'REACH', 'SPEND'], dtype=object)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_metric_comparison(df, metrics, group_col='column_name', env_col='env'):
    """
    USED AI to help create this function ! for looping between metric by column on index values
    Plots side-by-side bar charts comparing metrics between prod and dev environments.
    Parameters:
    - df: Combined DataFrame with both prod and dev data.
    - metrics: List of metric column names to compare (e.g. ['null_sum', 'value_sum']).
    - group_col: Column used for x-axis grouping (default: 'column_name').
    - env_col: Column that differentiates 'prod' and 'dev' (default: 'env').
    """
    # Unique values for the grouping (e.g., column names)
    x_vals = df[group_col].unique()

    # Split into prod and dev
    prod_df = df[df[env_col] == 'prod'].set_index(group_col)
    dev_df = df[df[env_col] == 'dev'].set_index(group_col)

    for metric in metrics:
        fig = go.Figure()

        # Add prod bars
        fig.add_trace(go.Bar(
            x=x_vals,
            y=prod_df.loc[x_vals][metric],
            name='Prod',
            marker_color='skyblue',
            text=prod_df.loc[x_vals][metric].round(2),
            textposition='auto'
        ))

        # Add dev bars
        fig.add_trace(go.Bar(
            x=x_vals,
            y=dev_df.loc[x_vals][metric],
            name='Dev',
            marker_color='salmon',
            text=dev_df.loc[x_vals][metric].round(2),
            textposition='auto'
        ))

        fig.update_layout(
            title=f'{metric.replace("_", " ").title()} Comparison by {group_col.replace("_", " ").title()}',
            xaxis_title=group_col.replace("_", " ").title(),
            yaxis_title=metric.replace("_", " ").title(),
            barmode='group',
            xaxis_tickangle=-45,
            height=500,
            width=1000,
            legend_title='Environment'
        )

        fig.show()

plot_metric_comparison(
    df=report_summary_combined,
    metrics=['null_sum', 'value_sum', 'distinct_type', 'duplicate_sum', 'percentage_of_null'])

## Producing a Table of the Difference
So far its not seemingly signficant but lets see how much there off

In [ ]:
prod_report_desc = pivoted_report_prod.describe()
dev_report_desc = report_dev_pivot.describe()
diff = prod_report_desc - dev_report_desc
display(diff)

METRIC_NAME,REPORTED_DATE,CAMPAIGN_ID,CHANNEL_REPORTED_REVENUE,CLICKS,CLICKTHROUGHS,CONVERSIONS,CPC,CPM,CTR,CVR,FREQUENCY,IMPRESSIONS,REACH,SPEND
count,0,0.000000e+00,0.0,1.000000,0.0,0.000000,0.0,0.0,-1.000000,0.0,0.0,1.000000,0.0,-1.000000
mean,0 days 00:00:00,-2.347737e+09,0.0,-0.076128,0.0,-0.001614,0.0,0.0,0.000732,0.0,0.0,-4.963415,0.0,0.067537
min,0 days 00:00:00,0.000000e+00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
25%,0 days 00:00:00,0.000000e+00,0.0,0.000000,0.0,0.000000,0.0,0.0,-0.001346,0.0,0.0,0.000000,0.0,-0.000526
50%,0 days 00:00:00,0.000000e+00,0.0,0.000000,0.0,0.000000,0.0,0.0,-0.000137,0.0,0.0,-4.000000,0.0,-0.039583
75%,0 days 00:00:00,-7.146910e+08,0.0,0.000000,0.0,0.000000,0.0,0.0,0.001125,0.0,0.0,-14.750000,0.0,0.117728
max,0 days 00:00:00,0.000000e+00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
std,NaN,6.678326e+08,0.0,-0.077403,0.0,0.000285,0.0,0.0,0.001575,0.0,0.0,-5.886838,0.0,0.092925


# Data Quality Check Dev vs Prod for Modeled
 Now looking into the same checks betwwen prod and dev in the data set for modeled data

In [ ]:
df_modeled_prod.head(2)

,REPORTED_DATE,COMPANY,MARKETING_CHANNEL,CAMPAIGN_ID,METRIC_NAME,METRIC_VALUE
0,2024-07-01,COMPANY_X,FACEBOOK_ADS,6281654088577,SECOND_ORDER_REVENUE,0.024461
1,2024-07-01,COMPANY_X,STACKADAPT,937310,SECOND_ORDER_REVENUE,80.976010


Get the shape and the data quality between both data sets to get the difference

In [ ]:
pivot_modeled_prod = df_pivot_data(df_modeled_prod)
dev_modeled_pivot = df_pivot_data(df_modeled_dev)
print("prod shape:", pivot_modeled_prod.shape)
print("dev shape:", dev_modeled_pivot.shape)
pivot_modeled_prod.head(10)

prod shape: (3250, 6)
dev shape: (3225, 6)


METRIC_NAME,REPORTED_DATE,COMPANY,MARKETING_CHANNEL,CAMPAIGN_ID,FIRST_ORDER_REVENUE,SECOND_ORDER_REVENUE
0,2024-07-01,COMPANY_X,ADWORDS,14731096318,766.434763,NaN
1,2024-07-01,COMPANY_X,ADWORDS,14734402526,0.000000,NaN
2,2024-07-01,COMPANY_X,ADWORDS,14740238797,0.000000,NaN
3,2024-07-01,COMPANY_X,ADWORDS,14744929637,0.000000,NaN
4,2024-07-01,COMPANY_X,ADWORDS,16878784202,3.060881,NaN
5,2024-07-01,COMPANY_X,ADWORDS,18137060772,3716.666764,7.071876
6,2024-07-01,COMPANY_X,ADWORDS,18137062041,0.000000,0.000000
7,2024-07-01,COMPANY_X,ADWORDS,18137062221,0.000000,0.000000
8,2024-07-01,COMPANY_X,ADWORDS,18138298060,0.000000,0.000000
9,2024-07-01,COMPANY_X,ADWORDS,18138298267,0.000000,0.000000


### - production had 25 more rows of data then the development how has influnced the ai model

*I saw that second modeled revenue was nan but I thought that adding both the first and second revenue it would make sense to add 0 to replace nan in the data set*

### - *First* step to check the quality before adding in a new caculated column.

Lets check first with the check summary and seeing the difference

Adding in the column of comparsion for the modeled data before checking the differences

- Now I am getting and adding a merge between the difference in data quality for the percentage of nulls , duplicates betwwen both enviorments in the data set and seeing the difference.

In [ ]:
modeled_prod_summary = check_data_summary(pivot_modeled_prod)
dev_modeled_summary = check_data_summary(dev_modeled_pivot)
display(modeled_prod_summary)
display(dev_modeled_summary)

,column_name,data_type,null_sum,value_sum,distinct_type,duplicate_sum,percentage_of_null
0,REPORTED_DATE,datetime64[ns],0,3250,19,3231,0.00
1,COMPANY,object,0,3250,1,3249,0.00
2,MARKETING_CHANNEL,object,0,3250,8,3242,0.00
3,CAMPAIGN_ID,int64,0,3250,172,3078,0.00
4,FIRST_ORDER_REVENUE,float64,57,3193,897,2352,1.75
5,SECOND_ORDER_REVENUE,float64,572,2678,777,2472,17.60


,column_name,data_type,null_sum,value_sum,distinct_type,duplicate_sum,percentage_of_null
0,REPORTED_DATE,datetime64[ns],0,3225,19,3206,0.00
1,COMPANY,object,0,3225,1,3224,0.00
2,MARKETING_CHANNEL,object,0,3225,8,3217,0.00
3,CAMPAIGN_ID,int64,0,3225,172,3053,0.00
4,FIRST_ORDER_REVENUE,float64,2,3223,1013,2211,0.06
5,SECOND_ORDER_REVENUE,float64,546,2679,1609,1615,16.93


## Graph to Show the difference between Prod and Dev Modeled Enviorments
- in the display there was a difference between  nulls where production had higher null percentage.

In [ ]:
modeled_prod_summary["env"] = "prod"
dev_modeled_summary["env"] = "dev"

modeled_summary_combined  = pd.concat([modeled_prod_summary, dev_modeled_summary], ignore_index=True)


plot_metric_comparison(
    df=modeled_summary_combined,
    metrics=['null_sum', 'value_sum', 'distinct_type', 'duplicate_sum', 'percentage_of_null'])

## Data Cleaning: Total Modeled Revenue
-- for nan+1 = nan in this instance have to fill na with 0 to get the approp results.

In [ ]:
pivot_modeled_prod["SECOND_ORDER_REVENUE"] = pivot_modeled_prod["SECOND_ORDER_REVENUE"].fillna(0)
dev_modeled_pivot["SECOND_ORDER_REVENUE"] = dev_modeled_pivot["SECOND_ORDER_REVENUE"].fillna(0)

pivot_modeled_prod["FIRST_ORDER_REVENUE"] = pivot_modeled_prod["FIRST_ORDER_REVENUE"].fillna(0)
dev_modeled_pivot["FIRST_ORDER_REVENUE"] = dev_modeled_pivot["FIRST_ORDER_REVENUE"].fillna(0)

pivot_modeled_prod["total_modeled_revenue"] = pivot_modeled_prod["FIRST_ORDER_REVENUE"]+pivot_modeled_prod["SECOND_ORDER_REVENUE"]
dev_modeled_pivot["total_modeled_revenue"] = dev_modeled_pivot["FIRST_ORDER_REVENUE"]+dev_modeled_pivot["SECOND_ORDER_REVENUE"]


Check to see total_modeled revenue :
- Note that beofre changing revenue this was the function applied:

In [ ]:
modeled_prod_summary = check_data_summary(pivot_modeled_prod)
dev_modeled_summary = check_data_summary(dev_modeled_pivot)
display(modeled_prod_summary)
display(dev_modeled_summary)

,column_name,data_type,null_sum,value_sum,distinct_type,duplicate_sum,percentage_of_null
0,REPORTED_DATE,datetime64[ns],0,3250,19,3231,0.0
1,COMPANY,object,0,3250,1,3249,0.0
2,MARKETING_CHANNEL,object,0,3250,8,3242,0.0
3,CAMPAIGN_ID,int64,0,3250,172,3078,0.0
4,FIRST_ORDER_REVENUE,float64,0,3250,897,2353,0.0
5,SECOND_ORDER_REVENUE,float64,0,3250,777,2473,0.0
6,total_modeled_revenue,float64,0,3250,990,2260,0.0


,column_name,data_type,null_sum,value_sum,distinct_type,duplicate_sum,percentage_of_null
0,REPORTED_DATE,datetime64[ns],0,3225,19,3206,0.0
1,COMPANY,object,0,3225,1,3224,0.0
2,MARKETING_CHANNEL,object,0,3225,8,3217,0.0
3,CAMPAIGN_ID,int64,0,3225,172,3053,0.0
4,FIRST_ORDER_REVENUE,float64,0,3225,1013,2212,0.0
5,SECOND_ORDER_REVENUE,float64,0,3225,1609,1616,0.0
6,total_modeled_revenue,float64,0,3225,1961,1264,0.0


# Difference for modeled data between prod and dev
- is noticable, how is the distribution different on revenue?

In [ ]:
prod_modeled_desc = pivot_modeled_prod.describe()
dev_modeled_desc =  dev_modeled_pivot.describe()
print('Prod Desc\n\n')
display(prod_modeled_desc)
print('dev Desc\n\n')
display(dev_modeled_desc)
diff_modeled = prod_modeled_desc - dev_modeled_desc
print('Difference between both\n\n')
diff_modeled

Prod Desc




METRIC_NAME,REPORTED_DATE,CAMPAIGN_ID,FIRST_ORDER_REVENUE,SECOND_ORDER_REVENUE,total_modeled_revenue
count,3250,3.250000e+03,3250.000000,3250.000000,3250.000000
mean,2024-07-09 22:48:13.292307712,1.197603e+16,274.340186,11.431998,285.772184
min,2024-07-01 00:00:00,1.624600e+05,0.000000,0.000000,0.000000
25%,2024-07-05 00:00:00,1.610270e+06,0.000000,0.000000,0.000000
50%,2024-07-10 00:00:00,6.267533e+11,0.000000,0.000000,0.000000
75%,2024-07-15 00:00:00,6.540410e+12,1.028391,0.000000,2.637498
max,2024-07-19 00:00:00,2.034528e+18,18736.344973,1786.658888,18743.467176
std,NaN,1.551229e+17,1124.704259,82.917938,1146.814269


dev Desc




METRIC_NAME,REPORTED_DATE,CAMPAIGN_ID,FIRST_ORDER_REVENUE,SECOND_ORDER_REVENUE,total_modeled_revenue
count,3225,3.225000e+03,3225.000000,3225.000000,3225.000000
mean,2024-07-09 21:07:12.000000256,1.143830e+16,199.997052,30.189300,230.186352
min,2024-07-01 00:00:00,1.624600e+05,0.000000,0.000000,0.000000
25%,2024-07-05 00:00:00,1.610270e+06,0.000000,0.000000,0.000000
50%,2024-07-10 00:00:00,6.267525e+11,0.000000,0.000000,0.779455
75%,2024-07-15 00:00:00,6.540410e+12,0.968003,2.241242,8.107326
max,2024-07-19 00:00:00,2.034528e+18,13267.321336,5418.819807,15231.682835
std,NaN,1.515899e+17,862.804311,255.420994,971.661955


Difference between both




METRIC_NAME,REPORTED_DATE,CAMPAIGN_ID,FIRST_ORDER_REVENUE,SECOND_ORDER_REVENUE,total_modeled_revenue
count,25,2.500000e+01,25.000000,25.000000,25.000000
mean,0 days 01:41:01.292307456,5.377284e+14,74.343134,-18.757302,55.585831
min,0 days 00:00:00,0.000000e+00,0.000000,0.000000,0.000000
25%,0 days 00:00:00,0.000000e+00,0.000000,0.000000,0.000000
50%,0 days 00:00:00,8.151260e+05,0.000000,0.000000,-0.779455
75%,0 days 00:00:00,0.000000e+00,0.060387,-2.241242,-5.469827
max,0 days 00:00:00,0.000000e+00,5469.023637,-3632.160919,3511.784341
std,NaN,3.532956e+15,261.899947,-172.503056,175.152314


## Capturing the table for insights on the difference:

In [ ]:
table_diff = diff_modeled.reset_index(inplace=True)
table_diff = diff_modeled.round(2).astype(str)


In [ ]:
# Getting column and rows
header = list(table_diff)
cells = [table_diff[col] for col in table_diff.columns]

# Using AI for table formatting
# Create table with better spacing
fig = go.Figure(data=[go.Table(
    columnwidth=[20] * len(header),  # Adjust column width as needed
    header=dict(
        values=header,
        fill_color='lightgrey',
        align='left',
        font=dict(size=10),
        height=32  # Row height for header
    ),
    cells=dict(
        values=cells,
         line_color='darkslategray',
        fill_color='white',
        align='left',
        font=dict(size=12),
        height=28  # Row height for cells
    )
)])

fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig.show()

# Comparison Between Dev's Modeled and Reported data

Now time to merge the correct data sets to compare the reported in dev to modled and vice verse for modeled starting with mergeing the data sets

In [ ]:
# getting the common overlapp between the data sets:
print(dev_modeled_pivot.columns)

report_dev_pivot.columns
shared_key = ['REPORTED_DATE',  'COMPANY',  'MARKETING_CHANNEL', 'CAMPAIGN_ID']


merged_development_df = pd.merge(dev_modeled_pivot, report_dev_pivot, on=shared_key, suffixes=("_reported","_modeled"))

merged_production_df = pd.merge(pivot_modeled_prod, pivoted_report_prod, on=shared_key, suffixes=("_reported","_modeled"))

Index(['REPORTED_DATE', 'COMPANY', 'MARKETING_CHANNEL', 'CAMPAIGN_ID',
       'FIRST_ORDER_REVENUE', 'SECOND_ORDER_REVENUE', 'total_modeled_revenue'],
      dtype='object', name='METRIC_NAME')


In [ ]:
merged_development_df.head(2)

METRIC_NAME,REPORTED_DATE,COMPANY,MARKETING_CHANNEL,CAMPAIGN_ID,FIRST_ORDER_REVENUE,SECOND_ORDER_REVENUE,total_modeled_revenue,CHANNEL_REPORTED_REVENUE,CLICKS,CLICKTHROUGHS,CONVERSIONS,CPC,CPM,CTR,CVR,FREQUENCY,IMPRESSIONS,REACH,SPEND
0,2024-07-01,COMPANY_X,ADWORDS,14731096318,517.614414,0.0,517.614414,1873.13,13.0,NaN,5.0,NaN,NaN,3.357646,3.521127,NaN,128.0,NaN,190.87
1,2024-07-01,COMPANY_X,ADWORDS,16878784202,1.755238,0.0,1.755238,0.00,15.0,NaN,NaN,NaN,NaN,3.772018,0.000000,NaN,97.0,NaN,99.09


## Now to add some caculations to see the impact of our model compared to the revenue given

### The three caculations are:
- difference between the modeled revenue vs reported
- percentage of difference
- ratio of the revenue modeled vs reported

In [ ]:
def add_diff_metrics(df, metric_reported, metric_modeled):
    diff_in_reveunue_modeled = f"{metric_reported}_diff_from_modeled"
    pct_diff_revenue = f"{metric_reported}_pct_diff_from_modeled"
    ratio_revenue_modeled = f"{metric_reported}_ratio_from_modeled"

    df[diff_in_reveunue_modeled] = df[metric_reported] - df[metric_modeled]
    df[pct_diff_revenue] = (df[diff_in_reveunue_modeled] / (df[metric_reported].replace(0, np.nan)))*100
    df[ratio_revenue_modeled ] = df[metric_reported] / (df[metric_modeled].replace(0, 1e-5))
    return df


# Heres the two df one for production and development

In [ ]:
development_df = add_diff_metrics(merged_development_df,"CHANNEL_REPORTED_REVENUE", 'total_modeled_revenue')

production_df = add_diff_metrics(merged_production_df,"CHANNEL_REPORTED_REVENUE",'total_modeled_revenue')

In [ ]:
production_df.head(10)

METRIC_NAME,REPORTED_DATE,COMPANY,MARKETING_CHANNEL,CAMPAIGN_ID,FIRST_ORDER_REVENUE,SECOND_ORDER_REVENUE,total_modeled_revenue,CHANNEL_REPORTED_REVENUE,CLICKS,CLICKTHROUGHS,...,CPM,CTR,CVR,FREQUENCY,IMPRESSIONS,REACH,SPEND,CHANNEL_REPORTED_REVENUE_diff_from_modeled,CHANNEL_REPORTED_REVENUE_pct_diff_from_modeled,CHANNEL_REPORTED_REVENUE_ratio_from_modeled
0,2024-07-01,COMPANY_X,ADWORDS,14731096318,766.434763,0.000000,766.434763,1873.130000,13.0,NaN,...,NaN,3.357646,3.521127,NaN,128.0,NaN,190.870000,1106.695237,59.082671,2.443952
1,2024-07-01,COMPANY_X,ADWORDS,16878784202,3.060881,0.000000,3.060881,0.000000,15.0,NaN,...,NaN,3.772018,0.000000,NaN,97.0,NaN,99.090000,-3.060881,NaN,0.000000
2,2024-07-01,COMPANY_X,ADWORDS,18137060772,3716.666764,7.071876,3723.738639,6775.150988,776.0,NaN,...,NaN,0.891494,1.984889,NaN,80109.0,NaN,4336.929657,3051.412349,45.038293,1.819449
3,2024-07-01,COMPANY_X,ADWORDS,20322881938,12057.813818,0.103114,12057.916932,15012.969488,873.0,NaN,...,NaN,0.543409,3.448121,NaN,160593.0,NaN,3453.604446,2955.052556,19.683332,1.245072
4,2024-07-01,COMPANY_X,ADWORDS,21048162675,3941.035877,0.000000,3941.035877,5497.302909,147.0,NaN,...,NaN,5.068900,2.129319,NaN,768.0,NaN,208.400000,1556.267032,28.309647,1.394888
5,2024-07-01,COMPANY_X,ADWORDS,21158648830,1205.375621,0.000000,1205.375621,2811.567231,90.0,NaN,...,NaN,1.779570,1.537691,NaN,1363.0,NaN,562.410000,1606.191610,57.127982,2.332524
6,2024-07-01,COMPANY_X,ADWORDS,21167876130,2.530382,3.451473,5.981855,2.846547,38.0,NaN,...,NaN,0.512438,0.003352,NaN,5628.0,NaN,147.770000,-3.135307,-110.144222,0.475864
7,2024-07-01,COMPANY_X,AMAZON_ADS,351487073423519,3432.996830,0.000000,3432.996830,3471.000000,332.0,NaN,...,NaN,NaN,NaN,NaN,12302.0,NaN,1301.180000,38.003170,1.094877,1.011070
8,2024-07-01,COMPANY_X,AMAZON_ADS,357015141883061,1677.760998,0.000000,1677.760998,2383.980000,523.0,NaN,...,NaN,NaN,NaN,NaN,109998.0,NaN,2161.000000,706.219002,29.623529,1.420929
9,2024-07-01,COMPANY_X,AMAZON_ADS,374998198609962,323.593396,0.000000,323.593396,379.000000,147.0,NaN,...,NaN,NaN,NaN,NaN,79229.0,NaN,400.070000,55.406604,14.619157,1.171223


In [ ]:
production_df.columns

Index(['REPORTED_DATE', 'COMPANY', 'MARKETING_CHANNEL', 'CAMPAIGN_ID',
       'FIRST_ORDER_REVENUE', 'SECOND_ORDER_REVENUE', 'total_modeled_revenue',
       'CHANNEL_REPORTED_REVENUE', 'CLICKS', 'CLICKTHROUGHS', 'CONVERSIONS',
       'CPC', 'CPM', 'CTR', 'CVR', 'FREQUENCY', 'IMPRESSIONS', 'REACH',
       'SPEND', 'CHANNEL_REPORTED_REVENUE_diff_from_modeled',
       'CHANNEL_REPORTED_REVENUE_pct_diff_from_modeled',
       'CHANNEL_REPORTED_REVENUE_ratio_from_modeled'],
      dtype='object', name='METRIC_NAME')

## Time to show it through Graphs: On
- modeled vs reported across add groups and there distrubtion for revenue
- percentaage of difference between production and development on projected revenue across time

- correlation matrixes to see in the reporting stage what was highly corrleated or not  

In [ ]:
def overall_revenue_comparison_with_labels(df):
    overall_agg = df.groupby('REPORTED_DATE').agg({
        'CHANNEL_REPORTED_REVENUE': 'sum',
        'total_modeled_revenue': 'sum',
    }).reset_index()

    # Format labels as strings with commas for thousands separators
    reported_labels = overall_agg['CHANNEL_REPORTED_REVENUE'].map(lambda x: f"{x:,.0f}")
    modeled_labels = overall_agg['total_modeled_revenue'].map(lambda x: f"{x:,.0f}")

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=overall_agg['REPORTED_DATE'],
        y=overall_agg['CHANNEL_REPORTED_REVENUE'],
        mode='lines+markers+text',
        name='Reported Revenue',
        line=dict(color='blue'),
        text=reported_labels,
        textposition='top center',
        textfont=dict(size=10, color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=overall_agg['REPORTED_DATE'],
        y=overall_agg['total_modeled_revenue'],
        mode='lines+markers+text',
        name='Modeled Revenue',
        line=dict(color='red'),
        text=modeled_labels,
        textposition='bottom center',
        textfont=dict(size=10, color='red')
    ))

    fig.update_layout(
        title='Total Modeled vs Reported Revenue Over Time',
        xaxis_title='Date',
        yaxis_title='Revenue',
        legend_title='Revenue Type',
        hovermode='x unified',
        height=600,
        margin=dict(t=50, b=50)
    )

    fig.show()



In [ ]:
overall_revenue_comparison_with_labels(production_df)
overall_revenue_comparison_with_labels(development_df)

## Based on this you can see the model is underperfoming over time
- Next step how much? and can we compare the percentage difference over time between both to see if production did better or worse at this than devolpment stage ?

# Comparsion bewteen Reported and Modeled data in Production

In [ ]:
# percengate diff between prod and development did the under preformance lesson over time

import plotly.graph_objects as go

def compare_pct_diff(product_agg, development_agg):
    # Convert percentage to formatted strings for labels
    product_labels = product_agg['CHANNEL_REPORTED_REVENUE_pct_diff_from_modeled'].map(lambda x: f"{x:.1f}%")
    dev_labels = development_agg['CHANNEL_REPORTED_REVENUE_pct_diff_from_modeled'].map(lambda x: f"{x:.1f}%")

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=product_agg['REPORTED_DATE'],
        y=product_agg['CHANNEL_REPORTED_REVENUE_pct_diff_from_modeled'],
        mode='lines+markers+text',
        name='Product % Diff',
        line=dict(color='blue'),
        text=product_labels,
        textposition='top center',
        textfont=dict(size=10, color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=development_agg['REPORTED_DATE'],
        y=development_agg['CHANNEL_REPORTED_REVENUE_pct_diff_from_modeled'],
        mode='lines+markers+text',
        name='Development % Diff',
        line=dict(color='orange'),
        text=dev_labels,
        textposition='bottom center',
        textfont=dict(size=10, color='orange')
    ))

    fig.update_layout(
        title='Revenue Model Percentage Difference: Product vs Development Over Time',
        xaxis_title='Date',
        yaxis_title='% Difference',
        yaxis=dict(
            tickformat=".1f",
            ticksuffix="%",
            range=[0, 100],
        ),
        legend_title='Group',
        hovermode='x unified',
        height=600
    )

    fig.show()


In [ ]:
product_agg = production_df.groupby('REPORTED_DATE')['CHANNEL_REPORTED_REVENUE_pct_diff_from_modeled'].mean().reset_index()
development_agg = development_df.groupby('REPORTED_DATE')['CHANNEL_REPORTED_REVENUE_pct_diff_from_modeled'].mean().reset_index()

compare_pct_diff(product_agg, development_agg)

#Using AI to build the visuals and tailoring to specfic concentrations
- Includes a graph of the actual vs the modled,
- model preformance by marketing channel
- time series over revenue
-

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

def scatter_actual_vs_modeled(df):
    """
    Why: Quickly shows how well predictions align with actuals — ideally, points lie near the diagonal.
    """
    x = df['CHANNEL_REPORTED_REVENUE']
    y = df['total_modeled_revenue']

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers',
        marker=dict(color='blue', size=6, opacity=0.6),
        name='Data Points'
    ))

    max_val = max(x.max(), y.max()) * 1.05
    fig.add_trace(go.Scatter(
        x=[0, max_val],
        y=[0, max_val],
        mode='lines',
        line=dict(color='red', dash='dash'),
        name='Perfect Prediction'
    ))

    fig.update_layout(
        title='Actual vs Modeled Revenue',
        xaxis_title='Actual Reported Revenue (USD $)',
        yaxis_title='Modeled Revenue (USD $)',
        xaxis=dict(
            range=[0, 30000],    # set x-axis from 0 to 30k
            tick0=0,             # start ticks at 0
            dtick=5000,          # ticks every 5,000
            tickprefix="$",
            separatethousands=True
        ),
        yaxis=dict(
            range=[0, max_val],
            tickprefix="$",
            separatethousands=True
        ),
        height=600,
        width=600,
        hovermode='closest'
    )
    fig.show()


def time_series_revenue(df):
    """
    Why: Visualizes how predictions and actual revenues trend over time. Helps spot consistent lags or spikes.
    """
    agg_df = df.copy()
    agg_df['REPORTED_DATE'] = pd.to_datetime(agg_df['REPORTED_DATE'])
    agg_df = agg_df.groupby('REPORTED_DATE').agg({
        'CHANNEL_REPORTED_REVENUE': 'sum',
        'total_modeled_revenue': 'sum'
    }).reset_index().sort_values('REPORTED_DATE')

    # Format labels with $ and comma separators
    reported_labels = agg_df['CHANNEL_REPORTED_REVENUE'].map(lambda x: f"${x:,.0f}")
    modeled_labels = agg_df['total_modeled_revenue'].map(lambda x: f"${x:,.0f}")

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=agg_df['REPORTED_DATE'],
        y=agg_df['CHANNEL_REPORTED_REVENUE'],
        mode='lines+markers+text',
        name='Reported Revenue',
        line=dict(color='blue'),
        text=reported_labels,
        textposition='top center',
        textfont=dict(size=10, color='blue')
    ))
    fig.add_trace(go.Scatter(
        x=agg_df['REPORTED_DATE'],
        y=agg_df['total_modeled_revenue'],
        mode='lines+markers+text',
        name='Modeled Revenue',
        line=dict(color='orange'),
        text=modeled_labels,
        textposition='bottom center',
        textfont=dict(size=10, color='orange')
    ))
    fig.update_layout(
        title='Revenue Over Time (USD $)',
        xaxis_title='Date',
        yaxis_title='Revenue (USD $)',
        hovermode='x unified',
        height=600
    )
    fig.show()


def error_over_time(df):
    """
    Why: Plots the residual (actual - predicted) over time. Useful to catch temporal bias or instability in the model.
    """
    agg_df = df.copy()
    agg_df['REPORTED_DATE'] = pd.to_datetime(agg_df['REPORTED_DATE'])
    agg_df['residual'] = agg_df['CHANNEL_REPORTED_REVENUE'] - agg_df['total_modeled_revenue']
    agg_df = agg_df.groupby('REPORTED_DATE')['residual'].mean().reset_index().sort_values('REPORTED_DATE')

    # Format residual labels with $ and comma
    residual_labels = agg_df['residual'].map(lambda x: f"${x:,.0f}")

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=agg_df['REPORTED_DATE'],
        y=agg_df['residual'],
        mode='lines+markers+text',
        name='Average Residual',
        line=dict(color='purple'),
        text=residual_labels,
        textposition='top center',
        textfont=dict(size=10, color='purple')
    ))
    fig.update_layout(
        title='Residual (Actual - Modeled) Over Time (USD $)',
        xaxis_title='Date',
        yaxis_title='Residual Revenue (USD $)',
        hovermode='x unified',
        height=500
    )
    fig.show()


def box_plot_error_by_channel(df):
    """
    Why: Highlights which channels the model performs well/poorly on by showing distribution of percentage errors.
    """
    fig = px.box(
        df,
        x='MARKETING_CHANNEL',
        y='CHANNEL_REPORTED_REVENUE_pct_diff_from_modeled',
        points='all',
        color='MARKETING_CHANNEL',
        title='Distribution of % Error by Marketing Channel',
        labels={'CHANNEL_REPORTED_REVENUE_pct_diff_from_modeled': '% Difference'},
        height=600
    )
    fig.update_layout(showlegend=False)
    fig.update_yaxes(range=[-200, 200])  # adjust this range as needed

    fig.show()


def rmse_by_channel(df):
    """Compare RMSE across marketing channels — sensitive to large prediction errors."""
    df = df.dropna(subset=['CHANNEL_REPORTED_REVENUE', 'total_modeled_revenue'])

    # Calculate RMSE per channel
    grouped = df.groupby('MARKETING_CHANNEL').apply(
        lambda g: np.sqrt(mean_squared_error(g['CHANNEL_REPORTED_REVENUE'], g['total_modeled_revenue']))
    ).reset_index(name='RMSE')

    # Sort by RMSE
    grouped = grouped.sort_values('RMSE', ascending=False)

    # Format RMSE labels with $ and 2 decimals
    rmse_labels = grouped['RMSE'].map(lambda x: f"${x:,.2f}")

    # Plot
    fig = px.bar(grouped,
                 x='MARKETING_CHANNEL',
                 y='RMSE',
                 title='RMSE by Marketing Channel (USD $)',
                 labels={'RMSE': 'Root Mean Squared Error (USD $)'},
                 text=rmse_labels,
                 height=500)
    fig.update_layout(xaxis_tickangle=-45)
    fig.update_traces(textposition='outside')
    fig.show()


def error_histogram(df):
    """
    Why: Checks if the prediction errors are symmetrically distributed or skewed — reveals systemic bias.
    """
    df = df.copy()
    df['error'] = df['CHANNEL_REPORTED_REVENUE'] - df['total_modeled_revenue']

    fig = px.histogram(
        df,
        x='error',
        nbins=50,
        title='Histogram of Prediction Errors (USD $)',
        labels={'error': 'Prediction Error (Actual - Modeled) (USD $)'}
    )
    fig.show()


def correlation_heatmap(df):
    """
    Why: Helps identify which numerical input features might influence revenue metrics
    by showing correlation between input features only (excludes target, IDs, and derived columns).
    """

    # Columns to exclude (identifiers, targets, derived errors)
    exclude_cols = [
        'REPORTED_DATE', 'COMPANY', 'MARKETING_CHANNEL', 'CAMPAIGN_ID',
        'FIRST_ORDER_REVENUE', 'SECOND_ORDER_REVENUE',
        'CHANNEL_REPORTED_REVENUE_diff_from_modeled',
        'CHANNEL_REPORTED_REVENUE_pct_diff_from_modeled',
        'CHANNEL_REPORTED_REVENUE_ratio_from_modeled',
        'error', 'pct_error', 'squared_error', 'residual', 'pct_residual'
    ]

    # Keep only relevant numeric input features
    corr_df = df.select_dtypes(include='number').drop(columns=exclude_cols, errors='ignore')

    # Compute correlation matrix
    corr_matrix = corr_df.corr().round(2)

    # Create heatmap
    fig = px.imshow(
        corr_matrix,
        text_auto=True,
        color_continuous_scale='RdBu_r',
        title='Correlation Heatmap of Input Features'
    )

    fig.update_layout(height=700)
    fig.show()


In [ ]:
def show_model_kpis(df):
    # Select only the relevant columns and drop NaNs
    df_clean = df[['CHANNEL_REPORTED_REVENUE', 'total_modeled_revenue']].copy()
    df_clean = df_clean.dropna()

    # Remove rows where actual is 0 to safely compute MAPE
    df_clean = df_clean[df_clean['CHANNEL_REPORTED_REVENUE'] != 0]

    actual = df_clean['CHANNEL_REPORTED_REVENUE']
    predicted = df_clean['total_modeled_revenue']

    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    # Avoid division by zero for MAPE:
    mape = np.mean(np.abs((actual - predicted) / actual.replace(0, np.nan))) * 100

    # Format with dollar signs and decimals
    mae_str = f"${mae:,.2f}"
    rmse_str = f"${rmse:,.2f}"
    mape_str = f"{mape:.2f}%"

    header = dict(values=["Metric", "Value"],
                  fill_color='lightgrey',
                  align='left')

    cells = dict(values=[["MAE", "RMSE", "MAPE"],
                  [mae_str, rmse_str, mape_str]],
                 line_color='darkslategray',
                 fill_color='white',
                 align='left')



    fig = go.Figure(data=[go.Table(header=header, cells=cells)])
    fig.update_layout(title="Model Performance KPIs")
    fig.show()

# Production Visuals of Actual vs Reported

In [ ]:
#Production visuals of modeled vs actual
scatter_actual_vs_modeled(production_df)
time_series_revenue(production_df)
error_over_time(production_df)
box_plot_error_by_channel(production_df)
error_histogram(production_df)
correlation_heatmap(production_df)
show_model_kpis(production_df)
rmse_by_channel(production_df)

/tmp/ipython-input-1177773918.py:159: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



## Based on the residuals being off i wanted to deep dive here on residuals :

In [ ]:
def percentage_residual_over_time(df, aggregate=True):
    """
    Plot percentage residuals ((actual - predicted) / actual * 100) over time.

    Parameters:
    - df: DataFrame with 'REPORTED_DATE', 'CHANNEL_REPORTED_REVENUE', 'total_modeled_revenue'
    - aggregate: If True, shows average % residual per day. If False, shows all raw % residuals as dots.
    """
    if aggregate:
        df_agg = df.groupby('REPORTED_DATE').agg({
            'CHANNEL_REPORTED_REVENUE': 'sum',
            'total_modeled_revenue': 'sum'
        }).reset_index()

        # Avoid divide-by-zero
        df_agg = df_agg[df_agg['CHANNEL_REPORTED_REVENUE'] != 0]

        df_agg['pct_residual'] = (
            (df_agg['CHANNEL_REPORTED_REVENUE'] - df_agg['total_modeled_revenue']) /
            df_agg['CHANNEL_REPORTED_REVENUE']
        ) * 100

        # Round to 1 decimal point for labels
        df_agg['pct_residual_label'] = df_agg['pct_residual'].round(1).astype(str) + '%'

        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=df_agg['REPORTED_DATE'],
            y=df_agg['pct_residual'],
            mode='lines+markers+text',
            text=df_agg['pct_residual_label'],
            textposition='top center',
            name='Percentage Residual',
            line=dict(color='red'),
            marker=dict(size=6)
        ))

    fig.update_layout(
        title='Revenue Percentage Residual (Actual - Predicted) Over Time',
        xaxis_title='Date',
        yaxis_title='Percentage Residual (%)',
        hovermode='x unified',
        height=500,
        template='plotly_white',
        yaxis=dict(range=[-100, 100], ticksuffix='%')
    )

    fig.show()



In [ ]:
percentage_residual_over_time(production_df)

# Development Visuals Stage of Model Performance

In [ ]:
scatter_actual_vs_modeled(development_df)
time_series_revenue(development_df)
error_over_time(development_df)
box_plot_error_by_channel(development_df)
rmse_by_channel(development_df)
error_histogram(development_df)
correlation_heatmap(development_df)
show_model_kpis(development_df)
percentage_residual_over_time(development_df)

/tmp/ipython-input-959367277.py:159: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

